## Manejo de variables categóricas e imputación de valores perdidos

En este Notebook os voy a dar información para aplicar técnicas para tratar con valores perdidos y transformar variables categóricas en variables numéricas para que todas las técnicas implementadas en Scikit-learn puedan utilizarse con toda la información disponible del problema a abordar.  Además, también se ofrece información sobre cómo utilizarlas en Pipelines y optimizar los híper-parámetros de todos los componentes de la misma de forma conjunta y cómo aplicar transformaciones diferentes a las variables en función de sus características. El guión es el siguiente:

* [Análsis de los datos](#0)
* [Imputación de valores perdidos](#1)
* [Transformación de variables categóricas a numéricas](#2)
* [Validación de modelos con Pipelines y ColumnTransformer](#3)

# Análisis de los datos <a class="anchor" id="0"></a>

Para conocer los tipos de datos almacenados en el DataFrame correspondiente al problema a abordar se puede visualizar la propiedad *dtypes*: *dataFrame.dtypes*.
* Para saber si la variable i-ésima es categórica o no podemos hacer: *dataFrame.dtypes[i]==object* o *dataFrame.dtypes[i]!=object*.
* Para obtener el número de categorías diferentes de una variable categórica podemos hacer: *dataFrame.nombreVariable.nunique()*

NOTA: las columnas cuyo tipo de dato sea *object* son posibles variables categóricas. La razón de que solamente sean posibles que si existieran valores perdidos en un atributo numérico y estuviera representado por un string también conllevaría que el tipo de la columna fuera *object*.

Para comprobar si las variables contienen valores perdidos o no podemos utilizar el método [*isnull*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isnull.html) de Pandas puesto que determina si los valores de una columna son nulos o no. 
* Si sumamos esa máscara obtendremos el número de valores perdidos.
* Si aplicamos *isnull* sin especificar ninguna variable y sumamos el resultado (la máscara booleana) obtenemos una *Serie* cuyo índice son los nombres de las variables y los valores son el número de valores nulos de cada una de ellas.
* Para obtener los índices de los ejemplos con valores nules en una variable se puede hacer: *dataFrame[dataFrame.variable.isnull()].index.tolist()*
* Podemos visualizar los ejemplos con valores nulos del siguiente modo: *dataFrame[dataFrame.isnull().any(axis=1)]*

# Imputación de valores perdidos <a class="anchor" id="1"></a>

Para imputar valores perdidos mediante la técnica de la imputación de la media (moda, mediana), Scikit-learn nos ofrece la clase [*SimpleImputer*](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) en la que el híper-parámetro *strategy* define la estrategia de imputación a utilizar:
* *mean*: asigna la media de los ejemplos en cada variable para los valores perdidos
* *median*: asigna la mediana de los ejemplos en cada variable para los valores perdidos
* *most_frequent*: asigna la moda de los ejemplos en cada variable para los valores perdidos

En caso de que existan valores perdidos en variables tanto en variables categóricas como numéricas debemos realizar la imputación adecuada a cada tipo de variable.

Para poder realizarlo, *Scikit-learn* provee la clase [*ColumnTransformer*](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) mediante la que se pueden aplicar determinadas transformaciones a determinadas variables. Para ello, en el híper-parámetro *transformers* se debe especificar una lista de tuplas en la que en cada tupla se determina el nombre de la transformación, la transformación a aplicar y las variables sobre las que aplicarla

    (nombreTransformación, transformación, ListaVariables)
    
Cabe destacar que la transformación a aplicar puede ser:
* Una técnica específica o una Pipeline en la que se encadenen varias técnicas a aplicar sobre las variables.
* El string *drop* para indicar que eliminen esas variables.
* El string *passthrough* para indicar que no haga nada con esas variables.

Otro híper-parámetro es *remainder* que especifica qué hacer con las variables que no se hayan especificado en el híper-parámetro *transformers*. Las opciones son los strings *drop* y *passthrough* explicados anteriormente. El valor por defecto es *drop* por lo que si hay variables que no se especifican en el híper-parámetro *transformers* serán eliminadas.
    
Una vez creado el objeto de la clase *ColumnTransformer* se pueden aplicar los métodos habituales de las técnicas de pre-procesamiento de datos: *fit*, *transform* y *fit_transform*.

Tras *ColumnTransformer*, el objeto que devuelve pasa a ser un array de Numpy. Por este motivo, en caso de que interese, se puede crear un DataFrame a partir de dicho objeto (y tener toda la información de nombres de variables por ejemplo). La único que hay que tener en cuenta es que las variables no estarán en el orden original sino en el orden en el que haya sido utilizadas por *ColumnTransformer*. Por tanto, el parámetro *columns* de *DataFrame* hay que asignarlo de forma adecuada (el *index* será el mismo que el del DataFrame original).

Otra forma de imputar valores perdidos que ofrece Scikit-learn es la imputación de valores perdidos de acuerdo a los vecinos más cercanos. La clase asociada a dicha técnica se llama [*KNNImputer*](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html). Los híper-parámetros más importantes de esta clase son:
* n_neighbors: valor entero que determina el número de vecinos a utilizar en la imputación. Por defecto es 5.
* weights: string (uniform o distance) que determina la forma de tener en cuenta a los vecinos más cercanos para realizar la imputación. Por defecto es uniform.
* copy: valor booleano que determina si realizar la imputación y devolver una copia del DataFrame (True, valor por defecto) o realizarla *in place* (False).

La implementación ofrecida en Scikit-learn de esta técnica tiene el problema de que las variables categóricas necesitan estar codificadas como numéricas con anterioridad a aplicarla (por usar los métodos de la clase KNN por debajo). Por ello, la tenemos que utilizar tras tener todas las variables como numéricas. Para ello, en las técnicas de transformación de variables categóricas a numéricas se debe hacer que en caso de que haya valores perdidos se transformen a *NaN* (es decir, *handle_missing='return_nan'* tal y como se explica posteriormente).

# Transformación de variables categóricas a numéricas <a class="anchor" id="2"></a>

Existe una librería en la que están implementadas numerosas técnicas de transformación de variables categóricas a numéricas. Esta librería se llama [*category_encoders*](https://pypi.org/project/category-encoders/) y la deberíamos instalar en caso de que no tengamos el entorno que os proporcionamos en la asigantura. Normalmente esta librería se importa con el nombre *ce*: *import category_encoders as ce*
    
En concreto vamos a utilizar los siguientes métodos de transformación de variables categóricas en numéricas:

* Codificación ordinal
* Codificación del conteo
* Codificación One-Hot
* Codificación binaria
* Codificación basada en la salida

Además de todas estas técnicas, también ofrece métodos avanzados como Catboost, el contraste de Helmert, el contraste polinomial, backward difference, etc... 

Las librerías Pandas y Scikit también implementan algunas de estas técnicas pero los procesos son más complicados y, en el caso de las implementaciones de Pandas no permiten incluir esta transformación dentro de una *Pipeline* al no tener implementados los métodos *fit* y *transform*.

## Codificación ordinal

La librería *category_encoders* ofrece la clase [*OrdinalEncoder*](http://contrib.scikit-learn.org/category_encoders/ordinal.html). Los híper-parámetros de esta clase son sencillos, debemos destacar cuatro:
* *cols*: permite establecer una lista con los nombres de las variables a transformar. Si se usa el valor por defecto, que es *None*, transforma todas las variables categóricas.
* *mapping*: es una lista de diccionarios que permitiría establecer manualmente los valores numéricos a asignar a las diferentes categorías de cada variable. 
    * Esta opción daría lugar a la técnica de transformación de variables categóricas a numericas conocida como *encoding labels*. En Scikit-learn se puede realizar esta transformación utilizando la clase [*LabelEncoder*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) y en Pandas mediante el método [*replace*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html).
* *handle_missing*: string que determina como tratar los valores perdidos a la otra de realizar la transformación. Las opciones son error (devuelve error), return_nan (devuelve NaN) y value (trata los valores perdidos como una categoría más y, por tanto, les asigna un nuevo valor numérico o -2 si no es en el proceso de entrenamiento).
* *handle_unknown*: string que determina cómo tratar nuevas categorías en test. Las opciones son las mismas que para el caso de valores perdidos pero en la opción valor transforma las nuevas caegorías a -1 (para diferenciarlas de los valores perdidos en caso de que haya al entrenar).
    
NOTA: Pandas permite realizar la transformación ordinal utilizando el método [*factorize*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html).

## Codificación del conteo

La librería *category_encoders* ofrece la clase [*CountEncoder*](http://contrib.scikit-learn.org/category_encoders/count.html). Los híper-parámetros de esta clase son similares a los de la clase anterior.

## Codificación One Hot

La librería *category_encoders* ofrece la clase [*OneHotEncoder*](http://contrib.scikit-learn.org/category_encoders/onehot.html) y los híper-parámetros son comunes a las clases anteriores. 
* Pandas permite realizar esta transformación mediante el método [*get_dummies*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html).
* Scikit-learn ofrece la clase [*LabelBinarizer*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html) para realizara esta transformación.

## Codificación binaria

La librería *category_encoders* ofrece la clase [*BinaryEncoder*](http://contrib.scikit-learn.org/category_encoders/binary.html) y los híper-parámetros son comunes a las clases anteriores. 

## Codificación basada en la salida

La librería *category_encoders* ofrece la clase [*TargetEncoder*](http://contrib.scikit-learn.org/category_encoders/targetencoder.html) y los híper-parámetros son comunes a las clases anteriores. Esta clase tiene un híper-parámetro llamado *smoothing* que determina si para asignar el valor se tiene en cuenta los valores de la variable a predecir de todos los ejemplos (independientemente de la categoría a sustituir):
* Regresión: la media de los valores (de la variable a predecir) que sean de la categoría a transformar se modifica teniendo en cuenta la media de los valores de la variable a predecir de todos los ejemplos. 
* Clasificación: la probabilidad de que los ejemplos que sean de la categoría a transformar sean de la clase 1 se modifica teniendo en cuenta la probabilidad de tener dicha clase en el conjunto de entrenamiento.

En ambos caso, cuanto mayor sea el valor del híper-parámetro *smoothing* más tenderá el valor transformado a la media de la variable a predecir de todos los ejemplos (o la probabilidad de tener ejemplos de la clase 1). 
* Un valor típico de *smoothing* para que se obtenga casi la media de los valores de la categoría a transformar es 0.0000001.

Además, como esta técnica utiliza información de la variable a predecir, al método *fit* se le debe pasar tanto los valores de los ejemplos en las variables de entrada (X) como en la de salida (y) para que pueda aprender los valores a asignar a cada categoría.

COMENTARIO GENERAL

Se pueden aplicar diferentes trasnformaciones a las variables en función de sus características. Por ejemplo, puede ser interesante aplicar una transformación ordinal a las variables con dos valores (o menos), una transformación *One hot* a las variables con pocos valores y una basada en la salida para las que tienen muchos y, de esta forma, evitar generar una matriz dispersa. Para ello, se debe utilizar *ColumnTransformer* sobre los grupos de variables de interés.

# Validación de modelos con Pipelines y ColumnTransformer <a class="anchor" id="3"></a>

Al igual que realizamos para un modelo de aprendizaje, existe la posibilidad de buscar la mejor configuración de toda la Pipeline (los mejores valores de sus híper-parámetros). Para ello, utilizamos la clase llamada *GridSearchCV* o *RandomizedSearchCV* de la librería *model_selection* que realiza tal proceso de forma automática.

Cuando las Pipelines contienen objetos *ColumnTransformer*, se pueden conseguir los mejores valores de los híper-parámetros utilizando *GridSearchCV*. Como estos objetos son mucho más complicados, para conocer los nombres de los híper-parámetros susceptibles de ser tratados podemos ejecutar la siguiente instrucción: *pipeline.get_params().keys()*

Algo útil para entender qué se hace en las Pipelines, y más cuando son complicadas como es el caso obtenido si alguno de los componentes es un *ColumnTransformer*, es mostrarlas visualmente. Para ello, existe el método [*set_config*](https://scikit-learn.org/stable/modules/generated/sklearn.set_config.html#sklearn.set_config) de Scikit-learn (nos permite mostrar visualmente objetos de muchas clases). El parámetro *display* determina si mostrar el resultado como una gráfica ('diagram') o como texto ('text'). Una vez determinada la forma de mostrar el resultado, al escribir el nombre de la variable donde se almacena el objeto de la clase de Scikit-learn éste se mostrará visualmente.

In [ ]:
# Importamos el método set_config
from sklearn import set_config

# Llamamos al método determinando que el resultado se muestre gráficamente
set_config(display="diagram")
# Determinamos el objeto sobre el que mostrar su flujo
modeloVisualizar  # Si hacéis click en el diagrama se muestran los detalles de cada componente